In [ ]:
import os
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
import random

from functionsFIMS import getAnalysisNumbers
from runDataClass import runData
from polyaClass import myPolya

In [ ]:
if __name__ == '__main__':
    runNos = getAnalysisNumbers()
    simData = runData(runNos[0])
    treenames = simData.getTreeNames()
    print(treenames)
    for name in treenames:
        simData.printColumns(name)

In [ ]:
if __name__ == '__main__':
    binWidth=3
    simData.plotAvalancheSize(binWidth)
    simData.plotAvalancheFits(binWidth)

    thresh = 10
    fitPolya = myPolya(24.701, 0.488)
    efficiency = fitPolya.calcEfficiencyErrs(thresh, 0.671, 0.095)
    print(f'{efficiency[0]:.3f} (+{efficiency[1]:.3f}, {efficiency[2]:.3f})')

In [ ]:
if __name__ == '__main__':
    simData.plotAvalanche2D(random.randint(0, 999), plotName='Random')

In [ ]:
TESTDATA = runData(1158)

In [ ]:
def getLargestRadius(xData, yData):
    """
    TODO
    """
    radius2 = xData**2 + yData**2
    maxRadius = math.sqrt(radius2.max())
    minRadius = math.sqrt(radius2.min())

    return maxRadius, minRadius

    

In [ ]:
def showIonStarts(runData):
    """
    TODO
    """
    allIons = runData.getDataFrame('ionData')
    posIons = allIons[allIons['Ion Charge']==1]
    
    bfIons = posIons[posIons['Final z']>=1]
    capIons = posIons[posIons['Final z']<1]


    bfMaxRad, bfMinRad = getLargestRadius(np.array(bfIons['Initial x'].values), np.array(bfIons['Initial y'].values))    
    bfCircle = plt.Circle(
        (0, 0), bfMaxRad, 
        facecolor='none', edgecolor='r', ls=':', lw=1, label='Backflow Region'
    )
    
    capMaxRad, capMinRad = getLargestRadius(np.array(capIons['Initial x'].values), np.array(capIons['Initial y'].values))   
    capCircle0 = plt.Circle(
        (0, 0), capMaxRad, 
        facecolor='none', edgecolor='g', ls=':', lw=1, label='Captured Region'
    )
    capCircle1 = plt.Circle(
        (0, 0), capMinRad, 
        facecolor='none', edgecolor='g', ls=':', lw=1
    )

    fig = plt.figure(figsize=(10, 10))
    fig.suptitle('Positive Ion Initial Locations')
    ax = fig.add_subplot(111)

    ax.scatter(
        bfIons['Initial x'], bfIons['Initial y'],
        label='BF Ions', c='r', marker='.'
    )
    ax.scatter(
        capIons['Initial x'], capIons['Initial y'],
        label='Captured Ions', c='g', alpha=.75, marker='.'
    )

    ax.add_patch(bfCircle)
    ax.add_patch(capCircle0)
    ax.add_patch(capCircle1)
    
    
    runData._plotAddCellGeometry(ax, 'xy')

    axLim = runData.getRunParameter('Pad Length')
    ax.set_xlim(-axLim, axLim)
    ax.set_ylim(-axLim, axLim)
    ax.legend()

    return fig

_ = showIonStarts(TESTDATA)